<a href="https://colab.research.google.com/github/idio4/task_vk/blob/main/VK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_ranking
!pip install optuna

In [2]:
import pandas as pd
import numpy as np
import tensorflow_ranking as tfr
import xgboost as xgb
from xgboost import plot_importance
import time
import optuna

In [3]:
from google.colab import files
uploaded = files.upload()

Saving test_df.csv to test_df (1).csv
Saving train_df.csv to train_df (1).csv


In [4]:
df = pd.read_csv("train_df.csv")
df_test = pd.read_csv("test_df.csv")

In [5]:
X_train = df.drop('target', axis=1)
y_train = df['target']
X_test = df_test.drop('target', axis=1)
y_test = df_test['target']

In [6]:
def objective(trial):
    params = {
        'objective': 'rank:ndcg',
        'eta': trial.suggest_loguniform('eta', 0.01, 0.2),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
    }


    dtrain = xgb.DMatrix(X_train, label=y_train)
    bst = xgb.train(params, dtrain, num_boost_round=100)

    dval = xgb.DMatrix(X_test)
    preds = bst.predict(dval)

    y_pred = preds.reshape(1, -1)
    y_true = y_test.values.reshape(1, -1)
    ndcg = tfr.keras.metrics.NDCGMetric()
    score = ndcg(y_true, y_pred).numpy()

    return score


In [7]:
#https://randomrealizations.com/posts/xgboost-parameter-tuning-with-optuna/
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='maximize', sampler=sampler)
tic = time.time()
while time.time() - tic < 300:
    study.optimize(objective, n_trials=1)

[I 2024-03-12 14:34:19,477] A new study created in memory with name: no-name-23310603-6622-4d16-9e7e-d374bfe4f52b
<ipython-input-6-f99431cfb0f7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'eta': trial.suggest_loguniform('eta', 0.01, 0.2),
<ipython-input-6-f99431cfb0f7>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
<ipython-input-6-f99431cfb0f7>:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
<ipython-input-6

In [9]:
best_params = study.best_params
dtrain = xgb.DMatrix(X_train, label=y_train)
bst = xgb.train(best_params, dtrain, num_boost_round=100)

In [10]:
dval = xgb.DMatrix(X_test)
preds = bst.predict(dval)
y_pred = preds.reshape(1,-1)
y_true = y_test.values.reshape(1,-1)
ndcg = tfr.keras.metrics.NDCGMetric()
ndcg(y_true, y_pred).numpy()

0.586037